# Creating an API

1. Create virtual environment using command: `python -m venv apienv`
2. Activate virtual environment using command:`apienv/Scripts/activate`
3. Install Djnago using command: `pip install django`
4. Intall Djnago rest framework using command: `pip install djangorestframework`
5. Create Django project using command: `django-admin startproject companyapi`
6. Navigate inside project directory using command: `cd companyapi`
6. Create an app using command: `python manage.py startapp api`
7. Open `settings.py`(companyapi) and add app `api` and `rest_framework` inside `INSTALLED_APPS`
8. Open `models.py`(api) and create models `Company` and `Employee` having one-one relationship using following lines of code
9. Create a file "serializers.py" inside `api` folder
10. Open `serializers.py`(api) and create serialzer using following lines of code
11. Open `views.py`(api) and create view set using following lines of code
12. Create a file "urls.py" inside `api` folder
13. Open `urls.py`(api) and create urls
14. Open `urls.py`(companyapi) and create url
15. Run command: `python manage.py makemigrations`
16. Run command: `python manage.py migrate`

In [ ]:
# Step8- companyapi/settings.py:

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'api', # app
    'rest_framework', # rest framework
]

In [ ]:
# Step9- api/models.py:

from django.db import models

# Create your models here.

TYPE=(
    ('IT','IT'),
    ('Non IT','Non IT'),
    ('Mobile Phones','Mobile Phones')
)

class Company(models.Model):
    company_id=models.AutoField(primary_key=True)
    name=models.CharField(max_length=50)
    location=models.CharField(max_length=50)
    about=models.TextField()
    type=models.CharField(max_length=100, choices=TYPE)
    added_date=models.DateTimeField(auto_now=True)
    active=models.BooleanField(default=True)
    
    def __str__(self):
        return self.name
    
DESIGNATION=(
    ('Manager', 'Manager'),
    ('Software Developer', 'sd'),
    ('Project leader', 'pl')
)
class Employee(models.Model):
    name=models.CharField(max_length=100)
    email=models.CharField(max_length=50)
    addres= models.CharField(max_length=200)
    phone=models.CharField(max_length=10)
    about=models.TextField()
    position=models.CharField(max_length=50, choices=DESIGNATION)
    company=models.ForeignKey(Company, on_delete=models.CASCADE)

In [ ]:
# Step11- api/serializers.py

from rest_framework import serializers
from api.models import Company, Employee

class CompanySerializer(serializers.HyperlinkedModelSerializer):
    company_id = serializers.ReadOnlyField() # exposing id
    class Meta:
        model=Company
        fields="__all__"
        
class EmployeeSerializer(serializers.HyperlinkedModelSerializer):
    id = serializers.ReadOnlyField() # exposing id
    class Meta:
        model=Employee
        fields="__all__"

In [ ]:
# Step12- api/views.py:

from django.shortcuts import render
from rest_framework import viewsets
from api.models import Company, Employee
from api.serializers import CompanySerializer, EmployeeSerializer

# Create your views here.
class CompanyViewSet(viewsets.ModelViewSet):
    queryset=Company.objects.all()
    serializer_class=CompanySerializer
    
class EmployeeViewSet(viewsets.ModelViewSet):
    queryset=Employee.objects.all()
    serializer_class=EmployeeSerializer

In [ ]:
# Step14- api/urls.py:

from django.urls import path, include
from rest_framework import routers

from api.views import CompanyViewSet, EmployeeViewSet


router=routers.DefaultRouter()
router.register(r'companies', CompanyViewSet)
router.register(r'employees', EmployeeViewSet)

urlpatterns = [
    path('v1/',include(router.urls)) 
]

# url will be like: v1/companies
# url will be like: v1/employees

In [ ]:
# Step15- companyapi/urls.py:

from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('api/', include("api.urls")), 
]

# complete url will be like: api/v1/companies
# complete url will be like: api/v1/employees

# Create a custom URL to get data

What we want here is we want to create an url that will fetch the record of all employee of a specific company as per our aboove example. Let's say our url for this will be like this: companies/{company_id}/employees

1. Open `views.py`(api) and update the code using following lines of code

In [ ]:
# Step1- api/views.py:

from django.shortcuts import render
from rest_framework import viewsets
from api.models import Company, Employee
from api.serializers import CompanySerializer, EmployeeSerializer
from rest_framework.decorators import action
from rest_framework.response import Response

# Create your views here.
class CompanyViewSet(viewsets.ModelViewSet):
    queryset=Company.objects.all()
    serializer_class=CompanySerializer
    
    # companies{company_id}/employees
    @action(detail=True, methods=['GET'])
    def employees(self, request, pk=None):
        try:
            company=Company.objects.get(pk=pk)
            emps=Employee.objects.filter(company=company)
            emps_serializer=EmployeeSerializer(emps, many=True, context={'request':request})
            return Response(emps_serializer.data)
        except Exception as e:
            return Response({'error':str(e)})
    
class EmployeeViewSet(viewsets.ModelViewSet):
    queryset=Employee.objects.all()
    serializer_class=EmployeeSerializer

# Making API read only and only can access JSON data

1. Open `settings.py`(companyapi) and add following settings

In [ ]:
# Step1- companyapi/settings.py:

REST_FRAMEWORK = {
    # Use Django's standard `django.contrib.auth` permissions,
    # or allow read-only access for unauthenticated users.
    'DEFAULT_PERMISSION_CLASSES': [ # for read only
        'rest_framework.permissions.DjangoModelPermissionsOrAnonReadOnly' 
    ],
    'DEFAULT_RENDERER_CLASSES':[ # for sending JSON data
        'rest_framework.renderers.JSONRenderer',
    ]
}